In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/chronbmm/sanskrit5-multitask

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/chronbmm/sanskrit5-multitask)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("chronbmm/sanskrit5-multitask")
model = AutoModelForSeq2SeqLM.from_pretrained("chronbmm/sanskrit5-multitask")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

# Mega Chunk input

In [4]:
# Paste ONE mega-chunk here (keep verse IDs)
mega_raw = """03,001.001 janamejaya uvāca
03,001.001a evaṃ dyūtajitāḥ pārthāḥ kopitāś ca durātmabhiḥ
03,001.001c dhārtarāṣṭraiḥ sahāmātyair nikṛtyā dvijasattama
03,001.002a śrāvitāḥ paruṣā vācaḥ sṛjadbhir vairam uttamam
03,001.002c kim akurvanta kauravyā mama pūrvapitāmahāḥ
03,001.003a kathaṃ caiśvaryavibhraṣṭāḥ sahasā duḥkham eyuṣaḥ
03,001.003c vane vijahrire pārthāḥ śakrapratimatejasaḥ
03,001.004a ke cainān anvavartanta prāptān vyasanam uttamam
03,001.004c kimāhārāḥ kimācārāḥ kva ca vāso mahātmanām

"""

## pre-processing text by removing IDs

In [5]:
import re

ID_RX = re.compile(r'^\s*\d{2},\d{3}\.\d{3}[a-z]?(?:\*[0-9_]+)?\s+')

def strip_ids_block(block: str) -> str:
    lines = []
    for ln in block.splitlines():
        ln = ln.strip()
        if not ln:
            continue
        lines.append(ID_RX.sub('', ln))
    return " ".join(lines)

chunk_text = strip_ids_block(mega_raw)
print("chars:", len(chunk_text))
print("first/last:", chunk_text.split()[0], "...", chunk_text.split()[-1])

chars: 384
first/last: janamejaya ... mahātmanām


In [3]:
chunk_text

"janamejaya uvāca evaṃ dyūtajitāḥ pārthāḥ kopitāś ca durātmabhiḥ dhārtarāṣṭraiḥ sahāmātyair nikṛtyā dvijasattama śrāvitāḥ paruṣā vācaḥ sṛjadbhir vairam uttamam kim akurvanta kauravyā mama pūrvapitāmahāḥ kathaṃ caiśvaryavibhraṣṭāḥ sahasā duḥkham eyuṣaḥ vane vijahrire pārthāḥ śakrapratimatejasaḥ ke cainān anvavartanta prāptān vyasanam uttamam kimāhārāḥ kimācārāḥ kva ca vāso mahātmanām kathaṃ dvādaśa varṣāṇi vane teṣāṃ mahātmanām vyatīyur brāhmaṇaśreṣṭha śūrāṇām arighātinām kathaṃ ca rājaputrī sā pravarā sarvayoṣitām pativratā mahābhāgā satataṃ satyavādinī vanavāsam aduḥkhārhā dāruṇaṃ pratyapadyata etad ācakṣva me sarvaṃ vistareṇa tapodhana śrotum icchāmi caritaṃ bhūridraviṇatejasām kathyamānaṃ tvayā vipra paraṃ kautūhalaṃ hi me vaiśaṃpāyana uvāca evaṃ dyūtajitāḥ pārthāḥ kopitāś ca durātmabhiḥ dhārtarāṣṭraiḥ sahāmātyair niryayur gajasāhvayāt vardhamānapuradvāreṇābhiniṣkramya te tadā udaṅmukhāḥ śastrabhṛtaḥ prayayuḥ saha kṛṣṇayā indrasenādayaś cainān bhṛtyāḥ paricaturdaśa rathair anuyayuḥ 

# tiny runners for S,L,M

In [6]:
import torch

@torch.inference_mode()
def run_task(prefix: str, text: str, max_new_tokens=768) -> str:
    enc = tokenizer([prefix + text], return_tensors="pt")
    enc = {k: v.to(model.device) for k, v in enc.items()}
    out = model.generate(**enc, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(out[0], skip_special_tokens=True)

## Split helpers (S+L and M)

---



In [11]:
def split_SL(s: str):
    # underscores + spaces both separate tokens for S/L
    return re.sub(r'[_\s]+', ' ', s).strip().split()

def parse_M(raw: str):
    """
    Returns a list of tuples: (surface, [morph_tags...])
    Example item: ('janamejaya', ['SNM','vac','SPs3In'])
    """
    bundles = raw.strip().split()         # keep per-token bundles
    out = []
    for b in bundles:
        parts = b.split('_')
        surface = parts[0]
        tags = parts[1:] if len(parts) > 1 else []
        out.append((surface, tags))
    return out

# running helpers and preview alignment

In [62]:
# --- 1) Run the three tasks on your current `chunk_text` ---
S_raw = run_task("S ", chunk_text, max_new_tokens=1024)  # sandhi split (more budget given)
L_raw = run_task("L ", chunk_text, max_new_tokens=768)   # lemma
M_raw = run_task("M ", chunk_text, max_new_tokens=768)   # morph bundle(s)

S = split_SL(S_raw)
L = split_SL(L_raw)
M_bundles = parse_M(M_raw)   # list of (surface, [tags...])

n = min(len(S), len(L), len(M_bundles))
print(f"lengths → S={len(S)}  L={len(L)}  M={len(M_bundles)}  → using {n}")

for i in range(min(60, n)):
    m_surface, m_tags = M_bundles[i]
    print(f"{i+1:02d} S='{S[i]}'  L='{L[i]}'  M_surface='{m_surface}'  M_tags={m_tags}")


lengths → S=56  L=99  M=55  → using 55
01 S='guṇān'  L='guṇa'  M_surface='guṇa'  M_tags=['PAM']
02 S='pārthasya'  L='pārtha'  M_surface='pārtha'  M_tags=['SGM']
03 S='saṃsmṛtya'  L='saṃsmṛ'  M_surface='saṃsmṛ'  M_tags=['Co']
04 S='duḥkha'  L='duḥkha'  M_surface='duḥkha'  M_tags=['Cp']
05 S='ārtāḥ'  L='ārta'  M_surface='ārta'  M_tags=['PNM']
06 S='parama'  L='parama'  M_surface='parama'  M_tags=['Cp']
07 S='āturāḥ'  L='ātura'  M_surface='ātura'  M_tags=['PNM']
08 S='akāmāḥ'  L='akāma'  M_surface='akāma'  M_tags=['PNM']
09 S='saṃnyavartanta'  L='saṃnivṛt'  M_surface='saṃnivṛt'  M_tags=['P3ImIn']
10 S='samāgamya'  L='samāgam'  M_surface='samāgam'  M_tags=['Co']
11 S='atha'  L='atha'  M_surface='atha'  M_tags=['']
12 S='pāṇḍavān'  L='pāṇḍava'  M_surface='pāṇḍava'  M_tags=['PAM']
13 S='nivṛtteṣu'  L='nivṛt'  M_surface='nivṛt'  M_tags=['PLPaM']
14 S='tu'  L='tu'  M_surface='tu'  M_tags=['']
15 S='paureṣu'  L='paura'  M_surface='paura'  M_tags=['PLM']
16 S='rathān'  L='ratha'  M_surface='rath

# save to csv

In [58]:
import csv
from pathlib import Path

chunk_label = "mega12"   # change per chunk: mega02, mega03, ...

out_rows = []
for i in range(n):
    m_surface, m_tags = M_bundles[i]
    out_rows.append({
        "chunk":     chunk_label,
        "token_idx": i + 1,
        "S":         S[i],
        "L":         L[i],
        "M_surface": m_surface,           # model’s surface in the M bundle
        "M_tags":    "|".join(m_tags),    # e.g., "SNM|vac|SPs3In"
    })

out_path = Path(f"tokens_SLM_{chunk_label}.csv")
with out_path.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["chunk","token_idx","S","L","M_surface","M_tags"])
    w.writeheader(); w.writerows(out_rows)

print(f"Saved {len(out_rows)} rows → {out_path}")

Saved 56 rows → tokens_SLM_mega13.csv


# aggregate the csv files based on header

In [ ]:
# --- Colab: Pick CSVs -> Merge -> Star split (fact + dim) ---

from google.colab import files
import os, re, io, pandas as pd

# 1) Upload: multi-select your 13 CSVs
uploaded = files.upload()  # choose all CSVs

# 2) Persist uploads to a folder (so we have filenames)
IN_DIR = "/content/csv_in"
os.makedirs(IN_DIR, exist_ok=True)
file_paths = []
for name, data in uploaded.items():
    path = os.path.join(IN_DIR, name)
    with open(path, "wb") as f:
        f.write(data)
    file_paths.append(path)

# 3) Natural sort so mega02, mega03, ... order is preserved
def natural_key(p):
    m = re.search(r"(\d+)(?=\.csv$)", os.path.basename(p))
    return int(m.group(1)) if m else 0

file_paths = sorted([p for p in file_paths if p.lower().endswith(".csv")], key=natural_key)
if not file_paths:
    raise FileNotFoundError("No CSVs selected.")

# 4) Robust CSV reader (strings only; trims BOM/whitespace in headers)
def read_csv_flex(path):
    try:
        df = pd.read_csv(path, dtype=str, keep_default_na=False, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(path, dtype=str, keep_default_na=False, encoding="utf-8")
    df.columns = [c.replace("\ufeff", "").strip() for c in df.columns]
    return df

# 5) Read all, unionize headers, track source file
dfs, all_cols, meta = [], set(), []
for p in file_paths:
    df = read_csv_flex(p)
    all_cols.update(df.columns)
    df["source_file"] = os.path.basename(p)
    meta.append({"file": os.path.basename(p), "rows": len(df), "cols": len(df.columns)})
    dfs.append(df)

# Unionize columns across files (fill missing with empty string)
all_cols = list(all_cols)
if "source_file" not in all_cols:
    all_cols.append("source_file")
dfs = [d.reindex(columns=all_cols, fill_value="") for d in dfs]

combined = pd.concat(dfs, ignore_index=True)

# 6) Assign 1-based id as the FIRST column
combined.insert(0, "id", range(1, len(combined) + 1))

# 7) Build dimension table: id, chunk, token_id, source_csv
#    (token_id is normalized from one of: token_idx, token_id, token)
def find_token_col(cols):
    for cand in ["token_idx", "token_id", "token"]:
        if cand in cols:
            return cand
    return None

token_col = find_token_col(combined.columns)
if token_col is None:
    raise KeyError("No token id column found in inputs (expected one of: token_idx, token_id, token).")

dim_cols = ["id"]
if "chunk" in combined.columns:
    dim_cols.append("chunk")
else:
    raise KeyError("No 'chunk' column found in inputs.")

dim_cols.append(token_col)  # will be renamed to token_id below
dim = combined[dim_cols + ["source_file"]].copy()
dim = dim.rename(columns={token_col: "token_id", "source_file": "source_csv"})

# 8) Build fact table: drop locator fields (chunk, token_id, source_csv/source_file)
fact = combined.copy()
for drop_col in ["chunk", token_col, "source_file", "source_csv"]:
    if drop_col in fact.columns:
        fact.drop(columns=[drop_col], inplace=True, errors="ignore")

# 9) Ensure formatting: make M_tags immediately follow M_surface in fact
cols = list(fact.columns)

def move_tags_after_surface(cols, surface="M_surface", tags="M_tags"):
    cols2 = cols[:]  # copy
    if surface in cols2 and tags in cols2:
        # remove both then reinsert surface followed by tags at surface's original index
        cols2.remove(surface)
        cols2.remove(tags)
        # find original surface index from original cols:
        surf_idx = cols.index(surface)
        # Rebuild: everything before original surface index (minus removed), then surface+tags, then the rest
        before = [c for c in cols if c in cols2][:surf_idx-0]  # safe; we’ll recompose next line anyway
        # Simpler: place id first always, then keep relative order while ensuring adjacency:
        # We'll craft final in a stable way below instead of the "before" trick.
    return cols2

# Stable reordering: ensure 'id' first, then maintain existing order,
# but if both M_surface and M_tags exist, place M_tags immediately after M_surface.
def reordered_fact_columns(cols):
    # id must be first
    rest = [c for c in cols if c != "id"]
    if "M_surface" in rest and "M_tags" in rest:
        # Move M_tags to immediately after M_surface
        rest.remove("M_tags")
        idx = rest.index("M_surface")
        rest.insert(idx + 1, "M_tags")
    return ["id"] + rest

fact = fact.reindex(columns=reordered_fact_columns(cols))

# 10) Save outputs
OUT_DIR = "/content"
fact_path = os.path.join(OUT_DIR, "fact_tokens_SLM.csv")
dim_path  = os.path.join(OUT_DIR, "dim_token_locator.csv")
summary_path = os.path.join(OUT_DIR, "combine_summary.csv")

fact.to_csv(fact_path, index=False)
dim.to_csv(dim_path, index=False)
pd.DataFrame(meta).to_csv(summary_path, index=False)

print(f"Done.\n- Fact:     {fact_path}\n- Dimension:{dim_path}\n- Summary:  {summary_path}")
print("\nFact preview:")
display(fact.head(5))
print("\nDimension preview:")
display(dim.head(5))

# 11) (Optional) one-click downloads
# from google.colab import files
# files.download(fact_path)
# files.download(dim_path)
# files.download(summary_path)

Saving tokens_SLM_mega01.csv to tokens_SLM_mega01 (1).csv
Saving tokens_SLM_mega02.csv to tokens_SLM_mega02 (1).csv
Saving tokens_SLM_mega03.csv to tokens_SLM_mega03 (1).csv
Saving tokens_SLM_mega04.csv to tokens_SLM_mega04 (1).csv
Saving tokens_SLM_mega05.csv to tokens_SLM_mega05 (1).csv
Saving tokens_SLM_mega06.csv to tokens_SLM_mega06 (1).csv
Saving tokens_SLM_mega07.csv to tokens_SLM_mega07 (1).csv
Saving tokens_SLM_mega08.csv to tokens_SLM_mega08 (1).csv
Saving tokens_SLM_mega09.csv to tokens_SLM_mega09 (1).csv
Saving tokens_SLM_mega10.csv to tokens_SLM_mega10 (1).csv
Saving tokens_SLM_mega11.csv to tokens_SLM_mega11 (1).csv
Saving tokens_SLM_mega12.csv to tokens_SLM_mega12 (1).csv
Saving tokens_SLM_mega13.csv to tokens_SLM_mega13 (1).csv
Done.
- Fact:     /content/fact_tokens_SLM.csv
- Dimension:/content/dim_token_locator.csv
- Summary:  /content/combine_summary.csv

Fact preview:


,id,L,S,M_surface,M_tags
0,1,janamejaya,janamejayaḥ,janamejaya,SNM
1,2,vac,uvāca,vac,SPs3In
2,3,evam,evam,evam,
3,4,dyūta,dyūta,dyūta,Cp
4,5,ji,jitāḥ,ji,PNPaM



Dimension preview:


,id,chunk,token_id,source_csv
0,1,mega01,1,tokens_SLM_mega01 (1).csv
1,2,mega01,2,tokens_SLM_mega01 (1).csv
2,3,mega01,3,tokens_SLM_mega01 (1).csv
3,4,mega01,4,tokens_SLM_mega01 (1).csv
4,5,mega01,5,tokens_SLM_mega01 (1).csv
